In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import pandas as pd
from matplotlib import rc
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import re, string
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import accuracy_score, classification_report
import joblib

rc('font', family='Malgun Gothic')
rc('axes', unicode_minus=False)

## 추가 코드

In [45]:
import pandas as pd
import random

df = pd.read_csv('DATA/챗봇_숙소문의.csv')

def shuffle_sentence(sentence):
    words = sentence.split()
    if len(words) > 1:
        random.shuffle(words) 
    return ' '.join(words)

def swap_words_within_same_label(df, label):
    same_label_questions = df[df['라벨링'] == label]['질문'].tolist()
    
    augmented_sentences = []
    for sentence in same_label_questions:
        words = sentence.split()
        if len(words) > 1:
            random.shuffle(words) 
        augmented_sentences.append(' '.join(words))
    
    return augmented_sentences

augmented_data = []

for label in df['라벨링'].unique():
    augmented_sentences = swap_words_within_same_label(df, label)
    
    for sentence in augmented_sentences:
        augmented_data.append({'질문': sentence, '라벨링': label})

augmented_df = pd.DataFrame(augmented_data)

combined_df = pd.concat([df, augmented_df])

if 'Unnamed: 2' in combined_df.columns:
    combined_df = combined_df.drop(columns=['Unnamed: 2'])
    
combined_df.to_csv('augmented_questions.csv', index=False, encoding='utf-8')

print(combined_df.head())

                   질문   라벨링
0   반려동물을 데리고 와도 되나요?  반려동물
1    애완동물을 동반할 수 있나요?  반려동물
2  반려동물과 함께 숙박 가능한가요?  반려동물
3     애완동물 입장이 허용되나요?  반려동물
4     반려동물 출입이 가능한지요?  반려동물


In [46]:
combined_df

,질문,라벨링
0,반려동물을 데리고 와도 되나요?,반려동물
1,애완동물을 동반할 수 있나요?,반려동물
2,반려동물과 함께 숙박 가능한가요?,반려동물
3,애완동물 입장이 허용되나요?,반려동물
4,반려동물 출입이 가능한지요?,반려동물
...,...,...
1130,있나요? 체크인 할 조기 수,체크인
1131,일찍 있나요? 체크인을 할 수,체크인
1132,허용되나요? 얼리 체크인이,체크인
1133,가능합니까? 체크인 일찍,체크인


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

label_encoder = LabelEncoder()
combined_df['라벨링'] = label_encoder.fit_transform(combined_df['라벨링'])

X_train, X_test, y_train, y_test = train_test_split(combined_df['질문'], combined_df['라벨링'], test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(sentences, max_length=128):
    return tokenizer(
        sentences.tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length
    )

X_train_tokenized = tokenize_data(X_train)
X_test_tokenized = tokenize_data(X_test)

y_train = y_train.values
y_test = y_test.values

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import warnings
warnings.filterwarnings('ignore')

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt",
        )
                
        input_ids = inputs['input_ids'].squeeze(0) 
        attention_mask = inputs['attention_mask'].squeeze(0)
        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = CustomDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = CustomDataset(X_test.tolist(), y_test.tolist(), tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator, 
    tokenizer=tokenizer
)

trainer.train()

In [74]:
def predict_question(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    predicted_label = label_encoder.inverse_transform([predicted_class])
    return predicted_label[0]

test_question = "숙소 주변에 주차할 공간이 있나요?"
print(f"질문: {test_question}, 예측된 라벨: {predict_question(test_question)}")

질문: 숙소 주변에 주차할 공간이 있나요?, 예측된 라벨: 주차


In [75]:
def predict_question(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    predicted_label = label_encoder.inverse_transform([predicted_class])
    return predicted_label[0]

test_question = "실내 취식이 가능한가요?"
print(f"질문: {test_question}, 예측된 라벨: {predict_question(test_question)}")

질문: 실내 취식이 가능한가요?, 예측된 라벨: 실내취식


In [76]:
def predict_question(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    predicted_label = label_encoder.inverse_transform([predicted_class])
    return predicted_label[0]

test_question = "체크인시간보다 일찍 가도 되나요?"
print(f"질문: {test_question}, 예측된 라벨: {predict_question(test_question)}")

질문: 체크인시간보다 일찍 가도 되나요?, 예측된 라벨: 얼리 체크인


In [10]:
data = pd.read_csv('augmented_questions.csv') 

In [31]:
model_path = '/Users/jeon-yewon/Downloads/코드 정리/3. 모델_chatbot/Model/질문 분류'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)


In [32]:
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['라벨링'])

label_encoder_path = 'Model/질문 분류/label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_path)

['Model/질문 분류/label_encoder.pkl']

In [33]:
data.head()

,질문,라벨링,encoded_labels
0,반려동물을 데리고 와도 되나요?,반려동물,4
1,애완동물을 동반할 수 있나요?,반려동물,4
2,반려동물과 함께 숙박 가능한가요?,반려동물,4
3,애완동물 입장이 허용되나요?,반려동물,4
4,반려동물 출입이 가능한지요?,반려동물,4


In [36]:
df['카테고리'].unique()

array(['반려동물', '세탁시설', '침구추가', '얼리 체크인', '지하철', '조리도구', '맛집', '세면도구',
       '수건제공', '조식', '주차', '실내취식', '관광지 추천', '짐 보관', '청소', '칫솔', 'OTT',
       '소음', '바다', '치안', '수건', '취식', '체크인', nan], dtype=object)

In [43]:
def predict_question(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    predicted_label = label_encoder.inverse_transform([predicted_class])
    
    return predicted_label[0]

test_question = "키우는 강아지를 데려가도 되나요?"
print(f"질문: {test_question}, 예측된 라벨: {predict_question(test_question)}")

질문: 키우는 강아지를 데려가도 되나요?, 예측된 라벨: 반려동물


In [41]:
test_question = "인원이 많아서 이불 추가 가능할까요?"
print(f"질문: {test_question}, 예측된 라벨: {predict_question(test_question)}")

질문: 인원이 많아서 이불 추가 가능할까요?, 예측된 라벨: 침구추가
